# Building a Responsible Real-time Web Search Application Workshop


## Introduction
We often find customers interested in integrating web search capabilities into their AI assistant use cases, especially when they need the most update-to-date information that's available online, but was not available to the LLMs when they were trained.

When creating applications like this without GenAI, you will have to define the logic of routing manually: when your application should search the web/when they should utilize the local knowledge etc. With [Bedrock Tool Use](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html), we can use LLMs to intelligently determine for us. You can define and implement the tools, such as database, Lambda function, or some other software, for your applications, In your code, you call the tool on the model's behalf. In this scenario, we define the tool implementation to be APIs. You then continue the conversation with the model by supplying a message with the result from the tool. Finally, the model generates a response for the original message that includes the tool results that you sent to the model.

[Amazon Bedrock Guardrails](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html) enables you to implement safeguards for your generative AI applications based on your use cases and responsible AI policies. You can create multiple guardrails tailored to different use cases and apply them across multiple foundation models (FM), providing a consistent user experience and standardizing safety and privacy controls across generative AI applications. For our web search use cases, we want to demonstrate how you can apply Guardrails on the search results 

In this workshop, we will demonstrate how to build a responsible real-time web search application utilizing Amazon Bedrock Tool Use feature, AWS Lambda functions and Amazon Bedrock Guardrails: 

- In Lab 0, we introduce the key concepts and services needed for this solution, and we provide an architectural diagram for you to understand how this application is built.
- In Lab 1, we will show you how to build the real-time web search application using the concepts and services we introduced in Lab 0.
- In Lab 2, we demonstrate how to integrate the Responsible AI to our application by adding Guardrails to the application we built in Lab 1. 

## Conceptual overview

### Function-Calling (Tool Use) with Converse API in Amazon Bedrock

In this section, we'll explore how to perform Function-Calling through the use of Tools with the Converse API for Amazon Bedrock.

The Converse or ConverseStream API is a unified structured text action for simplifying the invocations to Bedrock LLMs. It includes the possibility to define tools for implementing external functions that can be called or triggered from the LLMs.

Let's start by installing a few requirements for running this example. We only need to run this cell the first time.



In [2]:
!pip3 install -qU boto3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.35.14 which is incompatible.
sagemaker 2.215.0 requires protobuf<5.0,>=3.12, but you have protobuf 5.28.0 which is incompatible.


We can now import the relevant libraries.

In [3]:
import boto3
import json, sys
from datetime import datetime

print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.35.14


Let's also define a few variables. Make sure to adjust these parameters according to your needs.

In [4]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'
#modelId = 'cohere.command-r-plus-v1:0'
#modelId = 'cohere.command-r-v1:0'
#modelId = 'mistral.mistral-large-2402-v1:0'
print(f'Using modelId: {modelId}')

region = 'us-east-1'
print('Using region: ', region)

bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
    )

Using modelId: anthropic.claude-3-sonnet-20240229-v1:0
Using region:  us-east-1


We're now ready to define our tools through Python functions.

In our example, we will define a tool for simulating a weather forecast lookup tool (get_weather). Note in our example we're just returning a constant weather forecast to illustrate the concept, but you could make it fully functional by connecting any weather service API.

#### Define your tool

In [5]:
class ToolsList:
    #Define our get_weather tool function...
    def get_weather(self, city, state):
        #print(city, state)
        result = f'Weather in {city}, {state} is 70F and clear skies.'
        print(f'Tool result: {result}')
        return result

Let's structure our tools configuration for passing this information to our Converse API later. We have to clearly define the schema that our tools are expecting in the corresponding functions.

Note that we can also define specific configurations such as the tool choice, which allow us to either let the LLM choose automatically (auto), or overriding a fixed tool to be called always. You can check more information on this parameter in the Bedrock Converse API documentation.

In [6]:
#Define the configuration for our tool...
toolConfig = {'tools': [],
'toolChoice': {
    'auto': {},
    #'any': {},
    #'tool': {
    #    'name': 'get_weather'
    #}
    }
}

toolConfig['tools'].append({
        'toolSpec': {
            'name': 'get_weather',
            'description': 'Get weather of a location.',
            'inputSchema': {
                'json': {
                    'type': 'object',
                    'properties': {
                        'city': {
                            'type': 'string',
                            'description': 'City of the location'
                        },
                        'state': {
                            'type': 'string',
                            'description': 'State of the location'
                        }
                    },
                    'required': ['city', 'state']
                }
            }
        }
    })

Let's define a handy function for calling Bedrock with the Converse API.

#### Integrating tools with Converse API

In [7]:
#Function for caling the Bedrock Converse API...
def converse_with_tools(messages, system='', toolConfig=toolConfig):
    response = bedrock.converse(
        modelId=modelId,
        system=system,
        messages=messages,
        toolConfig=toolConfig
    )
    return response

We're now ready for setting up our orchestration flow. In this case, we'll make a first call to the LLM with the initial prompt from the user, and depending on the answer from the LLM we'll either call a tool (function calling) or end the interaction.

Note that in the case the LLM indicates that it wants to run a tool (function calling), it will give us the information required of tool name and arguments for us to run the relevant tool in our code; i.e. The LLMs cannot run the tools automatically.

In [8]:
#Function for orchestrating the conversation flow...
def converse(prompt, system=''):
    #Add the initial prompt:
    messages = []
    messages.append(
        {
            "role": "user",
            "content": [
                {
                    "text": prompt
                }
            ]
        }
    )
    print(f"\n{datetime.now().strftime('%H:%M:%S')} - Initial prompt:\n{json.dumps(messages, indent=2)}")

    #Invoke the model the first time:
    output = converse_with_tools(messages, system)
    print(f"\n{datetime.now().strftime('%H:%M:%S')} - Output so far:\n{json.dumps(output['output'], indent=2, ensure_ascii=False)}")

    #Add the intermediate output to the prompt:
    messages.append(output['output']['message'])

    function_calling = next((c['toolUse'] for c in output['output']['message']['content'] if 'toolUse' in c), None)

    #Check if function calling is triggered:
    if function_calling:
        #Get the tool name and arguments:
        tool_name = function_calling['name']
        tool_args = function_calling['input'] or {}
        
        #Run the tool:
        print(f"\n{datetime.now().strftime('%H:%M:%S')} - Running ({tool_name}) tool...")
        tool_response = getattr(ToolsList(), tool_name)(**tool_args) or ""
        if tool_response:
            tool_status = 'success'
        else:
            tool_status = 'error'

        #Add the tool result to the prompt:
        messages.append(
            {
                "role": "user",
                "content": [
                    {
                        'toolResult': {
                            'toolUseId':function_calling['toolUseId'],
                            'content': [
                                {
                                    "text": tool_response
                                }
                            ],
                            'status': tool_status
                        }
                    }
                ]
            }
        )
        #print(f"\n{datetime.now().strftime('%H:%M:%S')} - Messages so far:\n{json.dumps(messages, indent=2)}")

        #Invoke the model one more time:
        output = converse_with_tools(messages, system)
        print(f"\n{datetime.now().strftime('%H:%M:%S')} - Final output:\n{json.dumps(output['output'], indent=2, ensure_ascii=False)}\n")
    return

Now, we have everything setup and are ready for testing our function-calling bot.

Let's try with a few sample prompts, each one with different needs.

#### Testing Tool Use

In [9]:
prompts = [
    "What is the weather like in Queens, NY?",
    "What is the capital of France?"
]

for prompt in prompts:
    converse(
        system = [{"text": "You're provided with a tool that can get the weather information for a specific locations 'get_weather'; \
            only use the tool if required. Don't make reference to the tools in your final answer."}],
        prompt = prompt
)


23:35:41 - Initial prompt:
[
  {
    "role": "user",
    "content": [
      {
        "text": "What is the weather like in Queens, NY?"
      }
    ]
  }
]

23:35:43 - Output so far:
{
  "message": {
    "role": "assistant",
    "content": [
      {
        "toolUse": {
          "toolUseId": "tooluse_zuYEBloJRYWXmcVy921lFw",
          "name": "get_weather",
          "input": {
            "city": "Queens",
            "state": "NY"
          }
        }
      }
    ]
  }
}

23:35:43 - Running (get_weather) tool...
Tool result: Weather in Queens, NY is 70F and clear skies.

23:35:45 - Final output:
{
  "message": {
    "role": "assistant",
    "content": [
      {
        "text": "The weather in Queens, NY is currently pleasant with clear skies and a temperature around 70F."
      }
    ]
  }
}


23:35:45 - Initial prompt:
[
  {
    "role": "user",
    "content": [
      {
        "text": "What is the capital of France?"
      }
    ]
  }
]

23:35:45 - Output so far:
{
  "message": {

As we can see, the LLM decides whether or not to call the get_weather tool depending on the question.

You can further improve this example by playing with the system prompts. Also, you might want to switch the LLM and explore the differences.

Note that the Bedrock Converse API allow us to seamlessly use the same code and structure for working with multiple model providers. No changes are needed in the syntax of our interaction.

### Amazon Bedrock Guardrails - Examples using the Python SDK

Guardrails for Amazon Bedrock have multiple components which include Content Filters, Denied Topics, Word and Phrase Filters, and Sensitive Word (PII & Regex) Filters. For a full list check out the [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-create.html) 

#### Creating Guardrails

Guardrails for Amazon Bedrock have multiple components which include Content Filters, Denied Topics, Word and Phrase Filters, and Sensitive Word (PII & Regex) Filters. For a full list check out the [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-create.html) 

In [10]:
client = boto3.client('bedrock')

In [11]:
create_response = client.create_guardrail(
    name='fiduciary-advice',
    description='Prevents the our model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
    },
    blockedInputMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    blockedOutputsMessaging="""I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    tags=[
        {'key': 'purpose', 'value': 'fiduciary-advice-prevention'},
        {'key': 'environment', 'value': 'production'}
    ]
)

print(create_response)

{'ResponseMetadata': {'RequestId': '304b667c-4f23-4a6f-a1d7-d62e1a209864', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 06 Sep 2024 23:36:00 GMT', 'content-type': 'application/json', 'content-length': '172', 'connection': 'keep-alive', 'x-amzn-requestid': '304b667c-4f23-4a6f-a1d7-d62e1a209864'}, 'RetryAttempts': 0}, 'guardrailId': '12mi2ngynyh9', 'guardrailArn': 'arn:aws:bedrock:us-east-1:054464910056:guardrail/12mi2ngynyh9', 'version': 'DRAFT', 'createdAt': datetime.datetime(2024, 9, 6, 23, 36, 0, 261990, tzinfo=tzlocal())}


#### Getting a Guardrail, creating a version and listing all the versions and Drafts

In [12]:
#This will provide all the data about the DRAFT version we have
get_response = client.get_guardrail(
    guardrailIdentifier=create_response['guardrailId'],
    guardrailVersion='DRAFT'
)


In [13]:
# Now let's create a version for our Guardrail 
version_response = client.create_guardrail_version(
    guardrailIdentifier=create_response['guardrailId'],
    description='Version of Guardrail'
)

In [14]:
# To list the DRAFT version of all your guardrails, don’t specify the guardrailIdentifier field. To list all versions of a guardrail, specify the ARN of the guardrail in the guardrailIdentifier field.
list_guardrails_response = client.list_guardrails(
    guardrailIdentifier=create_response['guardrailArn'],
    maxResults=5)

print(list_guardrails_response)

{'ResponseMetadata': {'RequestId': '0afb7094-9ccc-4563-961f-e826db2ddd90', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 06 Sep 2024 23:36:01 GMT', 'content-type': 'application/json', 'content-length': '632', 'connection': 'keep-alive', 'x-amzn-requestid': '0afb7094-9ccc-4563-961f-e826db2ddd90'}, 'RetryAttempts': 0}, 'guardrails': [{'id': '12mi2ngynyh9', 'arn': 'arn:aws:bedrock:us-east-1:054464910056:guardrail/12mi2ngynyh9', 'status': 'VERSIONING', 'name': 'fiduciary-advice', 'description': 'Prevents the our model from providing fiduciary advice.', 'version': 'DRAFT', 'createdAt': datetime.datetime(2024, 9, 6, 23, 36, 0, 261990, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2024, 9, 6, 23, 36, 0, 347578, tzinfo=tzlocal())}, {'id': '12mi2ngynyh9', 'arn': 'arn:aws:bedrock:us-east-1:054464910056:guardrail/12mi2ngynyh9', 'status': 'CREATING', 'name': 'fiduciary-advice', 'description': 'Version of Guardrail', 'version': '1', 'createdAt': datetime.datetime(2024, 9, 6, 23, 36, 1, 5

#### Testing our Guardrail

In [19]:
# Build our request to Bedrock, we will test our second version

payload = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "How should I invest for my retirement? I want to be able to generate $5,000 a month"
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
response = bedrock.invoke_model(
    body = body_bytes,
    contentType = payload['contentType'],
    accept = payload['accept'],
    modelId = payload['modelId'],
    guardrailIdentifier = create_response['guardrailId'], 
    guardrailVersion ="1", 
    trace = "ENABLED"
)

# Print the response
response_body = response['body'].read().decode('utf-8')
print(json.dumps(json.loads(response_body), indent=2))

{
  "type": "message",
  "role": "assistant",
  "content": [
    {
      "type": "text",
      "text": "I can provide general info about Acme Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. "
    }
  ],
  "amazon-bedrock-trace": {
    "guardrail": {
      "input": {
        "12mi2ngynyh9": {
          "topicPolicy": {
            "topics": [
              {
                "name": "Fiduciary Advice",
                "type": "DENY",
                "action": "BLOCKED"
              }
            ]
          }
        }
      }
    }
  },
  "amazon-bedrock-guardrailAction": "INTERVENED"
}


# Architecture Diagram

Now we understand the key concepts, let's review the architecture for a responsible real-time web search application. 

